# Dependências

In [5]:
!pip install pyarrow

You should consider upgrading via the 'C:\Users\tield\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install fastparquet

You should consider upgrading via the 'C:\Users\tield\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install psycopg2

You should consider upgrading via the 'C:\Users\tield\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


# Bibliotecas

In [8]:
import numpy as np
import pandas as pd
import psycopg2 as postgres
import csv

# Funções

In [9]:
def connect():
    conn = None
    try:
        conn = postgres.connect(
            host="localhost",
            database="austin_test",
            user="postgres",
            password="root")
    except postgres.Error as e:
        print(e)
    return conn

In [10]:
def closeConnection(conn):
    sucess = False
    try:
        conn.close()
        sucess = True
    except postgres.Error as e:
        print(e)
    
    return sucess

In [11]:
def executeQuery(conn, sql):
    record = None
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        record = cur.fetchall()
        cur.close()
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()
    return record

In [12]:
def executeInsert(conn, sql):
    sucess = False
            
    try:
        #print(sql)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
        cur.close()
        sucess = True
    except postgres.Error as e:
        print(e)
        cur.execute("ROLLBACK")
        cur.close()

    return sucess

In [13]:
def extractTuplesLayerByBin(n, wgt):

  try:
    print('Extracting tuples of bin:', n, '...')
    #Step 0: Loading the ontology
    
    conn = connect()

    #file_name = './Austin/w07/Complete bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'.csv'
    #tuples = pd.read_csv(file_name)
    
    file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-p.parquet'
    tuples = pd.read_parquet(file_name)
    #tuples = tuples[['center_poi', 'context_poi', 'distance-m']]
    
    #layers = [1, 2, 3]
    layers = [1]
    for l in layers:

      layer_tuples = getTuplesFromLayer(conn, tuples, l)

      #Se há dados
      if(len(layer_tuples) > 0):
        
        #layer_tuples_file_name = './Austin/w07/Complete bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'.csv'          
        #layer_tuples.to_csv(layer_tuples_file_name, index=False)
        
        layer_tuples_file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'-p.parquet'          
        layer_tuples.to_parquet(layer_tuples_file_name, compression='brotli',  index = False)

    closeConnection(conn)               
  except Exception as e:
    #e
    print(str(e))

In [14]:
def getTuplesFromLayer(conn, tuples, layer):

  layer_tuples = pd.DataFrame(columns=tuples.columns.to_list())
  for id, row in tuples.iterrows():

    try:

      centerPoILayer = getClassLevel(conn, row['center_poi'].replace('\'', '\'\''))
      contextPoILayer = getClassLevel(conn, row['context_poi'].replace('\'', '\'\''))

      
      if (centerPoILayer[0][0] == contextPoILayer[0][0] and layer in centerPoILayer[0][0]):

        layer_tuples = layer_tuples.append(pd.DataFrame(
            [tuples.iloc[id].tolist()],
            columns=tuples.columns.to_list()), ignore_index = True
        )
    except Exception as e:
      e
      #print(str(e))
          
  return layer_tuples

In [21]:
def extractTuplesLayerByBinDisco(n, wgt):

  try:
    print('Extracting tuples of bin:', n, '...')
  
    conn = connect()

    file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-p.parquet'
    tuples = pd.read_parquet(file_name)
    #tuples = tuples[['center_poi', 'context_poi', 'distance-m']]
    
    layers = [1, 2, 3]
    #layers = [1]
    for l in layers:
      name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'-p.csv'
      csv_file = open(name, "w", newline='')
      writer = csv.writer(csv_file, delimiter=',')
      writer.writerow(["center_poi","context_poi","distance-m"])

      for id, row in tuples.iterrows():

        centerPoILayer = getClassLevel(conn, row['center_poi'].replace('\'', '\'\''))
        contextPoILayer = getClassLevel(conn, row['context_poi'].replace('\'', '\'\''))

        
        if (centerPoILayer[0][0] == contextPoILayer[0][0] and l == centerPoILayer[0][0]):
          line = tuples.iloc[id].tolist()
          print(line)
          writer.writerow(line)
          break
  
    csv_file.close()
    closeConnection(conn)               
  except Exception as e:
    #e
    print(str(e))

In [1]:
def extractTuplesLayerByBinDiscoML(n, wgt):

  try:
    print('Extracting tuples of bin:', n, '...')
  
    conn = connect()

    file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-p.parquet'
    tuples = pd.read_parquet(file_name)
    #tuples = tuples[['center_poi', 'context_poi', 'distance-m']]
    
    l = [1, 2, 3]
    
    name_l1 = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l[0])+'-p.csv'
    csv_file_l1 = open(name_l1, "w", newline='')
    writer_l1 = csv.writer(csv_file_l1, delimiter=',')
    writer_l1.writerow(["center_poi","context_poi","distance-m"])

    name_l2 = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l[1])+'-p.csv'
    csv_file_l2 = open(name_l2, "w", newline='')
    writer_l2 = csv.writer(csv_file_l2, delimiter=',')
    writer_l2.writerow(["center_poi","context_poi","distance-m"])

    name_l3 = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l[2])+'-p.csv'
    csv_file_l3 = open(name_l3, "w", newline='')
    writer_l3 = csv.writer(csv_file_l3, delimiter=',')
    writer_l3.writerow(["center_poi","context_poi","distance-m"])

    for id, row in tuples.iterrows():

      centerPoILayer = getClassLevel(conn, row['center_poi'].replace('\'', '\'\''))
      contextPoILayer = getClassLevel(conn, row['context_poi'].replace('\'', '\'\''))

      
      if (centerPoILayer[0][0] == contextPoILayer[0][0]):
        line = tuples.iloc[id].tolist()
        if (centerPoILayer[0][0] == l[0]):
          writer_l1.writerow(line)
        elif (centerPoILayer[0][0] == l[1]):
          writer_l2.writerow(line)
        elif (centerPoILayer[0][0] == l[2]):
          writer_l3.writerow(line)
        break
  
    csv_file_l1.close()
    csv_file_l2.close()
    csv_file_l3.close()
    closeConnection(conn)

  except Exception as e:
    #e
    print(str(e))

In [4]:
def getClassLevel(conn, className):
    sql = 'SELECT level FROM category WHERE name = \'' + className + '\';'
    result = executeQuery(conn, sql)
    return result

# Main

In [28]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

wgt = 0.5

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()/4))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = range(0, 1)

pool.starmap(extractTuplesLayerByBinDisco, [(n, wgt) for n in bins])
#pool.starmap(extractSentencesLayerByBin, [(n, wgt) for n in bins])

# Step 3: Don't forget to close
pool.close()
print("Process finish.")

1


True

In [18]:
n = 0
wgt = 0.5
l = 1


file_name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-p.parquet'
tuples = pd.read_parquet(file_name)

In [19]:
name = 'C:/Users/tield/Documents/Place2vec Experiments/Austin/w05/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'-p.csv'
csv_file = open(name, "w", newline='')
writer = csv.writer(csv_file, delimiter=',')
writer.writerow(["center_poi","context_poi","distance-m"])
line = tuples.iloc[0].tolist()
print(line)
writer.writerow(line)
csv_file.close()


['Shopping', 'Doctors', 12.9]


In [22]:
extractTuplesLayerByBinDisco(0, 0.5)

Extracting tuples of bin: 0 ...
['Shopping', 'Professional Services', 12.9]
